In [ ]:
import azureCognitiveSearch
import vec2Text
import vec2TextSentenceTransformerImage

import pickle 

In [ ]:
# Get all the files that will eventually be indexed
files = vec2Text.getFilesInDir(vec2Text.data_path)
print ('Total Files:', len(files))

# Load the cluster centers
with open(vec2Text.custer_center_file, 'rb') as pickle_in:
    clusterCenters = pickle.load(pickle_in)

In [ ]:
# Reset Index
azureCognitiveSearch.deleteIndex()
azureCognitiveSearch.createIndex()

In [ ]:
# Upload to Cognitive Search
max_batch_size = 100
documents = []

counter = 0
for file in files:
    curVec = vec2TextSentenceTransformerImage.imageEmbedding(file)
    vecText = ''
    for d in range(len(curVec)):
        vecText += vec2Text.convertFieldNumToString(d) + str(vec2Text.closest(clusterCenters[d], curVec[d])) + ' '

    documents.append({"@search.action": "upload","Id": vec2Text.stringToBase64(file), "Content": file,"VecText": vecText })

    counter += 1
    if len(documents) == max_batch_size:
        azureCognitiveSearch.uploadDocuments(documents)
        documents = []
        print ('Processed:', counter)

if len(documents) > 0:
    azureCognitiveSearch.uploadDocuments(documents)
    documents = []
    print ('Processed:', counter)
